# Datafolio

Para acceder a los datos que se desean analizar consulte  el siguiente [enlace.](https://datacatalog.worldbank.org/dataset/world-development-indicators) (Data & Resources)

In [1]:
#Importando librerías que tal vez pueda utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings
import ipywidgets as widgets
from ipywidgets import interact
import math
warnings.filterwarnings("ignore")  # Elimina las advertencias

El archivo `WDIData` contiene información de muchos indicadores de los países y regiones del mundo desde el año 1960, es notable que hay una gran cantidad de datos `NaN` ya que no ha sido posible recolectar la misma información en todos los países de igual manera.

El documento `WDISeries` contiene información más detallada del indicador presente en el archivo `WDIData`.

En la tabla `WDICountry` se muestra información relevante de cada uno de los países.

# Ejercicio 1

Haga una lista de todos los temas ('Topic') que hay en el archivo   `WDISeries`, seleccione un sub-tema, por ejemplo, el tema general puede ser 'Environment' y el sub-tema 'Land use' y agregue una columna a `WDISeries` cuyo valor sea el sub-tema elegido, si no es del sub-tema elegido ponga el valor `NaN`.

In [172]:
dfs=pd.read_csv('WDISeries.csv')
print(dfs['Topic'].unique())
dfs['Subtema']=np.where(dfs['Topic']=='Infrastructure: Communications', 'Infrastructure: Communications' ,None)

['Environment: Agricultural production' 'Environment: Land use'
 'Economic Policy & Debt: Balance of payments: Current account: Goods, services & income'
 'Economic Policy & Debt: Balance of payments: Capital & financial account'
 'Economic Policy & Debt: Balance of payments: Current account: Transfers'
 'Economic Policy & Debt: Balance of payments: Current account: Balances'
 'Economic Policy & Debt: Balance of payments: Reserves & other items'
 'Infrastructure: Communications' 'Financial Sector: Capital markets'
 'Economic Policy & Debt: Official development assistance'
 'Economic Policy & Debt: External debt: Debt outstanding'
 'Economic Policy & Debt: External debt: Debt ratios & other items'
 'Economic Policy & Debt: External debt: Net flows'
 'Economic Policy & Debt: External debt: Debt service'
 'World Bank, International Debt Statistics.'
 'Environment: Energy production & use' 'Environment: Emissions'
 'Environment: Biodiversity & protected areas'
 'Environment: Density & urba

# Ejercicio 2
Cree una nueva base de datos que contenga el histórico de todos los sucesos de su sub-tema por países y regiones. Llámela `sub-tema_df`. Trate esta nueva base de datos para que sea más legible y limpie, de ser necesario, la base.

In [177]:
df_WDI_Data=pd.read_csv('WDIData.csv')
df_WDI_Data=df_WDI_Data.drop('Unnamed: 65', axis=1)
lista_columns=df_WDI_Data.columns
lista_columns_titles=lista_columns[0:4]
lista_columns_unpivot= lista_columns[4:]
df_unpivot=pd.melt(df_WDI_Data, id_vars=lista_columns_titles, value_vars=lista_columns_unpivot, var_name='Año', value_name='Valor')
df_unpivot=df_unpivot[df_unpivot.Valor.notna()]
sub_tema_df=dfs[dfs.Subtema.notna()]
sub_tema_df=df_unpivot.merge(sub_tema_df,left_on='Indicator Code', right_on='Series Code')
dfcountry_series=pd.read_csv('WDICountry.csv')
sub_tema_df=sub_tema_df.merge(dfcountry_series,left_on='Country Code', right_on='Country Code')
sub_tema_df=sub_tema_df.drop(['Short definition','Unit of measure','Base Period','Other notes','Notes from original source','Related source links','Other web links'
                             ,'Related indicators','Unnamed: 20','PPP survey year','Unnamed: 30'], axis=1)
sub_tema_df=sub_tema_df.fillna('Sin Dato')
sub_tema_df=sub_tema_df[['Country Name', 'Country Code', 'Indicator Name_x', 'Indicator Code',
       'Año', 'Valor', 'Topic',
       'Periodicity', 'Aggregation method',
        'Source',  'License Type', 'Subtema', 'Short Name', 'Table Name', 'Long Name',
       '2-alpha code', 'Currency Unit', 'Region',
       'Income Group', 'WB-2 code', 'National accounts base year',
       'National accounts reference year', 'SNA price valuation',
       'Lending category', 'Other groups', 
       'Alternative conversion factor', 'Balance of Payments Manual in use',
       'External debt Reporting status', 'System of trade',
       'Government Accounting concept', 'IMF data dissemination standard',
    'Vital registration complete',  'Latest industrial data', 'Latest trade data']]

# Ejercicio 3
Divida la base de datos resultante del ejercicio anterior en dos, `sub-tema_C_df` y `sub-tema_R_df`, la primera tendrá la información por países y la segunda por regiones.

*Ayuda:* Use el archivo `WDICountry`para hacer la separación adecuada, la columna 'Regions' será de gran utilidad.

In [188]:
sub_tema_C_df=sub_tema_df[['Country Name', 'Country Code', 'Indicator Name_x', 'Indicator Code',
       'Año', 'Valor', 'Topic', 'Income Group','Subtema']]
sub_tema_R_df=sub_tema_df.groupby(['Region', 'Indicator Name_x', 'Indicator Code', 'Año', 'Topic',
       'Income Group', 'Subtema'])['Valor'].sum().astype('int64').reset_index(name='Valor')

# Ejercicio 4
La base de datos aún conserva valores un poco difíciles de leer, remplace los valores de la columna 'Indicator code' por una descripción un poco más amena para el lector. 

In [190]:
sub_tema_C_df

,Country Name,Country Code,Indicator Name_x,Indicator Code,Año,Valor,Topic,Income Group,Subtema
0,Arab World,ARB,Mobile cellular subscriptions,IT.CEL.SETS,1960,0.0,Infrastructure: Communications,Sin Dato,Infrastructure: Communications
1,Arab World,ARB,Mobile cellular subscriptions,IT.CEL.SETS,1965,0.0,Infrastructure: Communications,Sin Dato,Infrastructure: Communications
2,Arab World,ARB,Mobile cellular subscriptions,IT.CEL.SETS,1970,0.0,Infrastructure: Communications,Sin Dato,Infrastructure: Communications
3,Arab World,ARB,Mobile cellular subscriptions,IT.CEL.SETS,1975,0.0,Infrastructure: Communications,Sin Dato,Infrastructure: Communications
4,Arab World,ARB,Mobile cellular subscriptions,IT.CEL.SETS,1976,0.0,Infrastructure: Communications,Sin Dato,Infrastructure: Communications
...,...,...,...,...,...,...,...,...,...
90072,St. Martin (French part),MAF,Secure Internet servers,IT.NET.SECR,2015,4.0,Infrastructure: Communications,High income,Infrastructure: Communications
90073,St. Martin (French part),MAF,Secure Internet servers,IT.NET.SECR,2016,12.0,Infrastructure: Communications,High income,Infrastructure: Communications
90074,St. Martin (French part),MAF,Secure Internet servers,IT.NET.SECR,2017,9.0,Infrastructure: Communications,High income,Infrastructure: Communications
90075,St. Martin (French part),MAF,Secure Internet servers,IT.NET.SECR,2018,11.0,Infrastructure: Communications,High income,Infrastructure: Communications


# Ejercicio 5
Es aquí donde su creatividad será puesta a prueba, pues ya tiene una base de datos en buenas condiciones para representar gráficamente muchísima información, pero no se cohiba, si quiere puede agregar más columnas de los demás archivos para hacer más y graficos más específicos.

*Sugerencia:* Grafique la distribución de las variables obtenidas, el comparativo entre regiones, como también entre los países que más aportan, las diferencias entre los países con ingresos altos medios y bajos, no se le sugiere más porque usted debe ser el creativo.

Recuerde la información debe ser relevante, fácil de leer y puede emplear las diferentes librerías vistas hasta ahora para hacer de su datafolio el mejor.

In [253]:
from jupyter_dash import JupyterDash
import plotly_express as px
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from plotly import graph_objs as go
import dash_table
import seaborn as sns

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')
dfT=df[0:4]

colors = {
    'background': '#111111',
    'text': '#34495E',
    'rojo':'#FF1411',
    'negro': '#111111',
    'gris':'#D6DBDF',
    'gris_c':'#EBEDEF'
}
app = JupyterDash(__name__)

app.layout = html.Div([
    
    html.H1( 
        children='Visualización de Datos',
        style={
            'textAlign': 'center',
            'backgroundColor': colors['gris'],
             'color': colors['text'], 'marginTop': 0,'marginBottom': 0}
    )
,
     html.H2(children='Integrantes:',
              style={'marginBottom': 0, 'marginTop': 0})
,
    html.H3(children='Felipe Mejia',
           style={'marginBottom': 0,"margin-left": "120px", 'marginTop': 0}
           )
,
   
    html.H3(children='Michel Cespedes ',
           style={'marginBottom': 0,"margin-left": "120px", 'marginTop': 0})
,
    
    html.H3(children='Brayan Páez ',
             style={'marginBottom': 10,"margin-left": "120px", 'marginTop': 0})
,
 html.H4(children='A continuacion se mostrara la informacion con respecto al area elegida,en la cual podremos filtrar estos datos y visualizaciones con respecto a subtema, categoria de ingresos del pais,y rango de años en los cuales se desea ver el indicador elegido',
             style={'marginBottom': 30, 'marginTop': 30})
,
html.Div([
        html.Div([
            html.H3('Tema',style={'marginBottom': 10, 'marginTop': 10,'display': 'inline-block'}
            ),
        dcc.Dropdown(id='id_tema',
                     clearable=False,
            options=[{'label': i, 'value': i} for i in sub_tema_df['Indicator Name_x'].unique()],
            value='Mobile cellular subscriptions',
            multi=False)
                 ],style ={"margin-left": "60px","margin-right": "60px",'textAlign': 'center','width': '30%','display': 'inline-block'})
    ,
     html.Div([
              html.H3('Grupo de ingresos',style={'marginBottom': 10, 'marginTop': 10,'display': 'inline-block'}
                     ),
                dcc.RadioItems(id='item_g_ingresos',
                    options=[{'label': i, 'value': i} for i in sub_tema_df[sub_tema_df['Income Group'].notnull()]['Income Group'].sort_values().unique()],
                    value='Low income'
                              )
     ],style ={"margin-right": "60px","margin-left": "60px",'textAlign': 'center','width': '50%','display': 'inline-block'})   
])    
    
  
,
     html.H3('Año',style={'marginBottom': 30, 'marginTop': 30})
,
    
    dcc.RangeSlider(
        id='Slider_Año',
        min=(min(list(sub_tema_df['Año'].astype(int).sort_values().unique()))),
        max=max(list(sub_tema_df['Año'].astype(int).sort_values().unique())),
        step=1,
        value=[min(list(sub_tema_df['Año'].astype(int).sort_values().unique())), max(list(sub_tema_df['Año'].astype(int).sort_values().unique()))],
        marks={str(yr):str(yr) for yr in range (min(list(sub_tema_df['Año'].astype(int).sort_values().unique())), max(list(sub_tema_df['Año'].astype(int).sort_values().unique())),2)}
        
    )
,
    html.Div(id='Texto_Año', style={
            'textAlign': 'center'})
,
    
    html.H2(children='Datos',
           style={
            'textAlign': 'center',
            'backgroundColor': colors['gris'],
             'color': colors['text'], 'marginTop':20,'marginBottom': 10})
,
    
        html.Div([
            
 html.H4(children='Visualizaremos los datos obtenidos en las siguientes tablas',
             style={'marginBottom': 10, 'marginTop': 10})
            ,
    
            html.H3(children='Pais',
    
            
            
           style={
            'textAlign': 'center',
            'backgroundColor': colors['gris_c'],
             'color': colors['text'], 'marginTop': 0,'marginBottom': 10})
            ,
            
            
                            
 html.H4(children='Datos con la informacion diferenciada por paises',
             style={'marginBottom': 10, 'marginTop': 10}) ,
            
            dash_table.DataTable(
        id='id_tabla_pais',
        data=sub_tema_C_df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in sub_tema_C_df.columns],
        page_size=30,  #Cantidad de elementos por página
        style_table={'height': '300px', 'overflowY': 'auto'}, #Acá se crea el scroll
        style_header={'backgroundColor': 'rgb(30, 30, 30)'},
        style_cell={'backgroundColor': 'rgb(50, 50, 50)','color': 'white'}
                )
            ,
            
             html.H3(children='Región',
     
            
           style={
            'textAlign': 'center',
            'backgroundColor': colors['gris_c'],
             'color': colors['text'], 'marginTop': 50,'marginBottom': 10})
    ,
            html.H4(children='Datos con la informacion diferenciada por regiones del mundo',
             style={'marginBottom': 10, 'marginTop': 10})
            ,
            dash_table.DataTable(
        id='id_tabla_region',
        data=sub_tema_R_df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in sub_tema_R_df.columns],
        page_size=30,  #Cantidad de elementos por página
        style_table={'height': '300px', 'overflowY': 'auto'}, #Acá se crea el scroll
        style_header={'backgroundColor': 'rgb(30, 30, 30)'},
        style_cell={'backgroundColor': 'rgb(50, 50, 50)','color': 'white'}
                )
        ])
    
    
,   html.H1(children='Graficas',style={
            'textAlign': 'center',
            'backgroundColor': colors['gris'],
             'color': colors['text'], 'marginTop': 40,'marginBottom': 0})    
    

,   html.H2(children='Comparativa Incremento indicador con respecto al año anterior',style={
            'textAlign': 'center',
            'backgroundColor': colors['gris_c'],
             'color': colors['text'], 'marginTop': 40,'marginBottom': 0})
,
                                
 html.H4(children='Muestra el porcentaje de crecimiento o decrecimiento del indicador mostrado comparandolo con el año anterior',
             style={'marginBottom': 10, 'marginTop': 10}) ,
    
    dcc.Graph(id='graf_año')
    
,   html.H2(children='Indicador por Region',style={
            'textAlign': 'center',
            'backgroundColor': colors['gris_c'],
             'color': colors['text'], 'marginTop': 40,'marginBottom': 0})
,
                            
 html.H4(children='Muestra el indicador del tema elegido por region ',
             style={'marginBottom': 10, 'marginTop': 10}) ,
    dcc.Graph(id='graf_region')
 
,  
    
    html.H2(children='Top 10 Paises superior',style={
            'textAlign': 'center',
            'backgroundColor': colors['gris_c'],
             'color': colors['text'], 'marginTop': 20,'marginBottom': 10})
,
     html.H4(children='Muestra los 10 paises con el indicador mas alto ',
             style={'marginBottom': 10, 'marginTop': 10}) ,
    dcc.Graph(id='top_10_sup')
,   html.H2(children='Top 10 Paises inferior',style={
            'textAlign': 'center',
            'backgroundColor': colors['gris_c'],
             'color': colors['text'], 'marginTop': 40,'marginBottom': 0})
, html.H4(children='Muestra los 10 paises con el indicador mas bajo ',
             style={'marginBottom': 10, 'marginTop': 10}) ,
    dcc.Graph(id='top_10_inf')
])

@app.callback(
    Output('Texto_Año', 'children'),
    [Input('Slider_Año', 'value')])
def update_output(value):
    return '{}'.format(value)



@app.callback(
    Output('id_tabla_pais', 'data'),
    [Input('id_tema', 'value')
    ,Input('item_g_ingresos', 'value')
    ,Input('Slider_Año', 'value')])
def update_tabla(value1,value2,value3):
    df=sub_tema_C_df[(sub_tema_C_df['Indicator Name_x']==value1)&(sub_tema_C_df['Income Group']==value2)&(sub_tema_C_df['Año'].astype(int)>=value3[0])&(sub_tema_C_df['Año'].astype(int)<=value3[1])]
    return df.to_dict('records')



@app.callback(
    Output('id_tabla_region', 'data'),
    [Input('id_tema', 'value')
    ,Input('item_g_ingresos', 'value')
    ,Input('Slider_Año', 'value')])
def update_tabla(value1,value2,value3):
    df=sub_tema_R_df[(sub_tema_R_df['Indicator Name_x']==value1)&(sub_tema_R_df['Income Group']==value2)&(sub_tema_R_df['Año'].astype(int)>=value3[0])&(sub_tema_R_df['Año'].astype(int)<=value3[1])]
    return df.to_dict('records')



@app.callback(
    Output('graf_año', 'figure'),
    [Input('id_tema', 'value')
    ,Input('item_g_ingresos', 'value')
    ,Input('Slider_Año', 'value')])
def update_graf_año(value1,value2,value3):
    tema=value1
    Año_min=value3[0]
    Año_max=value3[1]
    Grupo_ingresos=[value2]
    sub_tema_df_cel=sub_tema_C_df[(sub_tema_C_df['Indicator Name_x']==tema)&(sub_tema_C_df['Income Group'].isin(Grupo_ingresos))]
    df_indi_año=sub_tema_df_cel.groupby(['Año'])['Valor'].sum().astype('int64').reset_index(name='Total')
    df_indi_año['Año']=(df_indi_año['Año'].astype(int))
    df_indi_año['Año_1']=df_indi_año['Año'].astype(int)+1
    df_indi_año=df_indi_año[['Año','Total']].merge(df_indi_año[['Año_1','Total']],left_on='Año',right_on='Año_1')
    df_indi_año=df_indi_año[['Año','Total_x','Total_y']]
    df_indi_año['Percent']=((df_indi_año['Total_x']/df_indi_año['Total_y'] -1)).round(3).astype(float)
    df_indi_año=df_indi_año[df_indi_año['Percent']!=np.inf]
    df_indi_año=df_indi_año[df_indi_año.Percent.notna()]
    df_indi_año=df_indi_año[ (df_indi_año['Año']>=Año_min) & (df_indi_año['Año']<=Año_max)]
    df_indi_año['Año']=df_indi_año['Año'].astype(int)
    fig = px.line(df_indi_año, x='Año', y='Percent')
#     fig.add_bar(df_indi_año, x='Año', y='Total_x')
    fig.update_layout(yaxis_tickformat = '%')
    return fig

@app.callback(
    Output('graf_region', 'figure'),
    [Input('id_tema', 'value')
    ,Input('item_g_ingresos', 'value')
    ,Input('Slider_Año', 'value')])
def update_graf_region(value1,value2,value3):
    tema=value1
    Año_min=value3[0]
    Año_max=value3[1]
    Grupo_ingresos=[value2]
    sub_tema_df_cel=sub_tema_R_df[(sub_tema_df['Indicator Name_x']==tema)&(sub_tema_R_df['Income Group'].isin(Grupo_ingresos))]
    sub_tema_df_cel['Año']=sub_tema_df_cel['Año'].astype(int)
    sub_tema_df_cel=sub_tema_df_cel[(sub_tema_df_cel['Año']>=Año_min)&(sub_tema_df_cel['Año']<=Año_max)]
    df_indi_año=sub_tema_df_cel.groupby(['Region'])['Valor'].sum().astype('int64').reset_index(name='Total')
    fig = px.bar(df_indi_año, x='Region', y='Total')
    return fig



@app.callback(
    Output('top_10_sup', 'figure'),
    [Input('id_tema', 'value')
    ,Input('item_g_ingresos', 'value')
    ,Input('Slider_Año', 'value')])
def update_top_10_sup(value1,value2,value3):
    tema=value1
    Año_min=value3[0]
    Año_max=value3[1]
    Grupo_ingresos=[value2]
    sub_tema_df_cel=sub_tema_C_df[(sub_tema_df['Indicator Name_x']==tema)&(sub_tema_C_df['Income Group'].isin(Grupo_ingresos))&(sub_tema_C_df['Año'].astype(int)>=Año_min)&(sub_tema_C_df['Año'].astype(int)<=Año_max)]
    sub_tema_df_cel['Año']=sub_tema_df_cel['Año'].astype(int)
    df_indi_año=sub_tema_df_cel.groupby(['Country Name','Año'])['Valor'].sum().astype('int64').reset_index(name='Total')
    df_indi_año=df_indi_año.groupby(['Country Name'])['Total'].sum().astype('int64').reset_index(name='Total').sort_values(by='Total', ascending=False).head(10)
    fig = px.bar(df_indi_año, x='Country Name', y='Total')
    return fig



@app.callback(
    Output('top_10_inf', 'figure'),
    [Input('id_tema', 'value')
    ,Input('item_g_ingresos', 'value')
    ,Input('Slider_Año', 'value')])
def update_top_10_inf(value1,value2,value3):
    tema=value1
    Año_min=value3[0]
    Año_max=value3[1]
    Grupo_ingresos=[value2]
    sub_tema_df_cel=sub_tema_C_df[(sub_tema_df['Indicator Name_x']==tema)&(sub_tema_C_df['Income Group'].isin(Grupo_ingresos))&(sub_tema_C_df['Año'].astype(int)>=Año_min)&(sub_tema_C_df['Año'].astype(int)<=Año_max)]
    sub_tema_df_cel['Año']=sub_tema_df_cel['Año'].astype(int)
    df_indi_año=sub_tema_df_cel.groupby(['Country Name','Año'])['Valor'].sum().astype('int64').reset_index(name='Total')
    df_indi_año=df_indi_año.groupby(['Country Name'])['Total'].sum().astype('int64').reset_index(name='Total').sort_values(by='Total', ascending=True).head(10)
    fig = px.bar(df_indi_año, x='Country Name', y='Total')
    return fig





if __name__ == '__main__':
    app.run_server(debug=True)
   

Dash app running on http://127.0.0.1:8050/


# Observaciones:
El datafolio debe:
```
- Enfatizar la narración y visualización de datos sobre la explicación con mucho texto.
- Ser de fácil lectura para la audiencia  y debe dar a entender rápidamente la esencia del trabajo.
- Ser atractivo y accesible para audiencias 'no técnicas'.
```